In [2]:
import requests #para obtener los datos de la API
import pandas as pd #para estructurarlos
import numpy as np

Fase 1: Extracción de datos de películas

Objetivo: Extraer 100 películas del endpoint

In [ ]:
# 1. Definir el endpoint de la API
url_api = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

# 2. Realizar la solicitud HTTP
try:
    response = requests.get(url_api)
    response.raise_for_status() # Lanza una excepción si la solicitud falla
    data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Error al obtener los datos de la API: {e}")
    data = []

# 3. Almacenar los datos en un DataFrame (DataFrame)
if data:
    # La API devuelve una lista de diccionarios
    df_peliculas = pd.DataFrame(data)

    # Solo nos interesan las primeras 100 películas
    df_peliculas = df_peliculas.head(100)

    # 4. Seleccionar y renombrar las columnas según el enunciado
    # Las columnas del enunciado son: Título, Año de lanzamiento, Duración (en minutos), Género, Contenido para adultos
    

    df_peliculas = df_peliculas.rename(columns={
        'titulo': 'titulo',
        'año': 'ano_lanzamiento',
        'duracion': 'duracion_minutos',
        'genero': 'genero',
        'adultos': 'contenido_adulto'
    })

    # Seleccionamos solo las columnas que necesitamos (ya que nuestro API tiene más)
    df_peliculas = df_peliculas[['titulo', 'ano_lanzamiento', 'duracion_minutos', 'genero', 'contenido_adulto']]

    print("Extracción y preparación de datos completada.")
    print(df_peliculas.head())
else:
    print("No se pudieron obtener datos para el DataFrame.")

Extracción y preparación de datos completada.
                  titulo  ano_lanzamiento  duracion_minutos  genero  \
0          The Godfather             1972               175  Crimen   
1  The Godfather Part II             1974               202  Crimen   
2           Pulp Fiction             1994               154  Crimen   
3           Forrest Gump             1994               142   Drama   
4        The Dark Knight             2008               152  Acción   

   contenido_adulto  
0             False  
1             False  
2              True  
3             False  
4             False  


In [4]:
df_peliculas

,titulo,ano_lanzamiento,duracion_minutos,genero,contenido_adulto
0,The Godfather,1972,175,Crimen,False
1,The Godfather Part II,1974,202,Crimen,False
2,Pulp Fiction,1994,154,Crimen,True
3,Forrest Gump,1994,142,Drama,False
4,The Dark Knight,2008,152,Acción,False
...,...,...,...,...,...
95,La vita è bella,1997,116,Drama,False
96,Requiem for a Dream,2000,102,Drama,True
97,Memento,2000,113,Thriller,True
98,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False


Fase 2: Creación de la base de datos e inserción de datos

In [5]:
pip install mysql-connector-python pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import mysql.connector
from mysql.connector import errorcode
from mysql.connector import Error # <-- Se necesita para el primer except

# 1. Configuración de la conexión 
nombre_bd = "Adalab_pelis"
cnx = None # Inicializamos cnx

try:
    # Intento de conexión inicial sin especificar la DB para crearla
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',
    )
    print('Conexión exitosa para crear DB')
    
    mycursor = cnx.cursor()
    query = f"CREATE DATABASE IF NOT EXISTS {nombre_bd}"
    mycursor.execute(query)
    print("Base de datos creada/asegurada.")
    
    # Cerramos la conexión inicial (sin DB)
    mycursor.close()
    cnx.close()
    
    # 2. Conexión a la Base de Datos Creada 
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',
        database=nombre_bd # <-- Conectamos directamente a la DB
    )
    mycursor = cnx.cursor()

    # 4. Creación de la Tabla 'peliculas'
    TABLES = {}
    TABLES['peliculas'] = (
        "CREATE TABLE `peliculas` ("
        "  `id_pelicula` INT NOT NULL AUTO_INCREMENT,"
        "  `titulo` VARCHAR(255) NOT NULL,"
        "  `ano_lanzamiento` YEAR NOT NULL,"
        "  `duracion_minutos` INT NOT NULL,"
        "  `genero` VARCHAR(100) NOT NULL,"
        "  `contenido_adulto` VARCHAR(3) NOT NULL,"
        "  PRIMARY KEY (`id_pelicula`)"
        ") ENGINE=InnoDB")

    try:
        print("Creando tabla peliculas...")
        mycursor.execute(TABLES['peliculas'])
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("La tabla 'peliculas' ya existe, continuando...")
        else:
            print(f"Error al crear la tabla: {err}")
    
    # 5. Inserción de Datos desde el DataFrame (df_peliculas)
    
    # Preparamos la sentencia SQL de inserción
    insert_query = (
        "INSERT INTO peliculas "
        "(titulo, ano_lanzamiento, duracion_minutos, genero, contenido_adulto) "
        "VALUES (%s, %s, %s, %s, %s)"
    )
    
    # Convertimos el DataFrame a una lista de tuplas para la inserción
    records_to_insert = [
        (row['titulo'], row['ano_lanzamiento'], row['duracion_minutos'], row['genero'], row['contenido_adulto'])
        for index, row in df_peliculas.iterrows()
    ]
    
    # Ejecutamos la inserción
    mycursor.executemany(insert_query, records_to_insert)
    
    # Confirmar los cambios
    cnx.commit()
    print(f"Inserción completada. {mycursor.rowcount} filas insertadas.")

except Error as e: # <-- Maneja errores de la primera conexión y creación de DB
    print(f"Error general en la conexión/DB: {e}")
    
finally:
    if cnx and cnx.is_connected():
        mycursor.close()
        cnx.close()
        print("Conexión a MySQL cerrada.")

Conexión exitosa para crear DB
Base de datos creada/asegurada.
Creando tabla peliculas...
La tabla 'peliculas' ya existe, continuando...
Inserción completada. 100 filas insertadas.
Conexión a MySQL cerrada.
